# Banco de Dados 2

Preparação corra 🏃‍♂️‍➡️ os seguintes códigos:

```python
# Chamando bibliotecas - banco de dados
import sqlite3 
from contextlib import closing
```

```python
# Criando um banco de dados (Create)
conexão = sqlite3.connect("agenda.db")
cursor = conexão.cursor()
cursor.execute(''' create table agenda(nome text,telefone text) ''')
cursor.execute('''insert into agenda (nome, telefone) values(?,?)''', ("Nilo","7788-1432"))
conexão.commit()
cursor.close()
conexão.close
```

```python
# Inserindo registros (Create)
dados = [("João","98901-0109"),
("André","98902-8900"),
("Maria","97891-3321")]
conexão = sqlite3.connect("agenda.db")
cursor = conexão.cursor()
cursor.executemany(''' 
    insert into agenda(nome,telefone)
    values(?,?)
 ''',dados)
conexão.commit()
cursor.close()
conexão.close()
```

```python
# Pesquisando um banco de dados (Recover)
from contextlib import closing 
with sqlite3.connect('agenda.db') as conexão:
    with closing(conexão.cursor()) as cursor:
        cursor.execute("select * from agenda")
        while True:
            resultado = cursor.fetchone()
            if resultado is None:
                break
            print(f'Nome: {resultado[0]} \nTelefone: {resultado[1]}')
```

## Consultando registros

Até agora, não fomos além do que poderíamos ter feito com simples arquivos de texto. A facilidade de um sistema de banco de dados começa a aparecer quando precisamos procurar e alterar dados. Ao trabalharmos com arquivos, essas operações devem ser implementadas em nossos programas, mas, com o SQLite, podemos realizá - las usando comandos SQL. Primeiro, vamos utilizar uma variação do comando `select` para mostrar apenas alguns registros, implementando uma seleção de registros com base em uma pesquisa. Pesquisas em SQL são feitas com a cláusula `where` . Vejamos o comando SQL , que seleciona todos os registros da agenda, cujo o nome seja igual a "Nilo".

```sql
select * from agenda where nome = "Nilo"
```

 Veja que apenas acrescentamos a cláusula `where` após o nome da tabela. O critério de seleção ou de pesquisa deve ser escrito como uma expressão, no caso `nome = "Nilo"`. O programa 4 mostra com essa modificação (em que o `=` em SQL seria equivalente ao `==` do Python).

```python
# Programa 4 - Consulta com filtro de seleção
with sqlite3.connect("agenda.db") as conexão:
    with closing(conexão.cursor()) as cursor:
        cursor.execute("select * from agenda where nome = 'Nilo'")
        while True:
            resultado = cursor.fetchone()
            if resultado is None:
                break
            print(f"Nome: {resultado[0]}\nTelefone: {resultado[1]}")
```

Ao executarmos o Programa 4 devemos ter apenas um: resultado:

```
Nome: Nilo
Telefone: 7788-1432
```


Veja que escrevemos 'Nilo' entre apóstrofos. Aqui, podemos usar um pouco do já sabemos sobre strings em Python e escrever:

```sql
cursor.execute('select * from agenda nome = "Nilo"')
```

Ou seja, poderíamos trocar as aspas por apóstrofos ou ainda usar aspas triplas:

```sql
cursor.execute(""" select * from agenda where nome = "Nilo" """)
```

 No caso do nosso exemplo, o nome 'Nilo' é uma constante e não há problemas em escrevê-lo diretamente em nosso comando `select`. No entanto, caso o  nome a filtrar viesse de uma varável, ficaríamos tentados a escrever um programa, como:
 
 ```python
nome = input("Nome a selecionar: ")
with sqlite3.connect("agenda.db") as conexão:
    with closing(conexão.cursor()) as cursor:
        cursor.execute(f'select * from agenda where nome = "{nome}"')
        while True:
            resultado = cursor.fetchone()
            if resultado is None:
                break
            print(f"Nome: {resultado[0]}\nTelefone: {resultado[1]}")
```
Execute o programa com vários valores: Nilo, João e Maria. Experimentamos também com um nome que não existe. A cláusula `where` funciona de forma parecida a um filtro. Imagine que o comando `select` cria uma lista e que a expressão lógica definida no where é avaliada para cada elemento. Quando o resultado dessa avaliação é verdadeiro, a linha é copiada para uma outra lista, a lista de resultados, retornada pela nossa consulta.

Veja que o programa funciona relativamente muito bem, exceto quando nada encontramos e o programa termina sem dizer muita coisa. Nós vamos corrigir esse problema logo a seguir, mas execute o programa mais uma vez e digite a seguinte sequência como nome:

```
X" or "1" = "1
```

Resultado:

```
Nome: Nilo
Telefone: 7788-1432

Nome: João
Telefone: 98901-0109

Nome: André
Telefone: 98902-8900

Nome: Maria
Telefone: 97891-3321

```

Surpreso com o resultado? Esse é o motivo de não utilizarmos variáveis em  nossos consultas. Esse tipo de vulnerabilidade é um exemplo de SQLInjection, um ataque bem conhecido. Isso acontece porque o comando SQL resultante é:

```sql
SELECT * FROM agenda WHERE nome = X" or "1" = "1
```

Para evitar esse tipo de ataque, sempre utilize parâmetros com valores variáveis.

O `or` da linguagem SQL funciona de forma semelhante ao `or` do Python. Dessa forma, nossa entrada de dados foi modificada por um valor digitado no programa. Esse tipo de erro é muito grave e pode ficar muito tempo em nossos programas sem ser percebido. Isso acontece porque a consulta é uma string como outra qualquer, e o valor passado para o método `execute` é a string resultante. Dessa forma, o valor digitado pelo usuário pode introduzir elementos que nós não desejamos. Os operadores relacionais `and` e `not` funcionam exatamente como em Python, e você também pode usá-los em expressões SQL.

Para não cairmos nesse tipo de armadilha, utilizaremos sempre parâmetros em nossa consultas:

```python
# Programa 5: Consulta utilizando parâmetros
nome = input("Nome a selecionar: ")
with sqlite3.connect("agenda.db") as conexão:
    with closing(conexão.cursor()) as cursor:
        cursor.execute('SELECT * FROM agenda WHERE nome = ?',(nome,))
        x = 0
        while True:
            resultado = cursor.fetchone()
            if resultado is None:
                if x == 0:
                    print("Nada encontrado.")
                break
            print(f"Nome: {resultado[0]}\nTelefone: {resultado[1]}") 
            x +=1 
```


Agora utilizamos um parâmetro como fizemos antes para inserir nossos registros. Um detalhe importante é que escrevemos (nome,), repare a vírgula após `nome`. Esse detalhe importante é que apresentamos, pois o segundo parâmetro do método `execute` é uma tupla, e, em Python, tuplas com apenas um elemento são escritas com uma vírgula após o primeiro valor. Veja também que utilizamos a variável `x` para contar quantos resultados obtivemos. Como o método `fetchone` retorna `None` quando todos registros foram recebidos, verificamos se `x == 0`. Isso acontece, pois valores diferentes de 0 no Python são conhecidos como verdadeiros e `None`,0,"",{},[] como falsos.

***Exercício 3*** - Escreva um programa que realize consultas do banco de dados preços.db, criado no Exercício 1. O programa deve perguntar o nome do produto e listar seu preço.

*Obs.: Exercício 1 - Faça um programa que crie o banco de dados preços.db com a tabela preços para armazenar uma lista de preços de venda de produtos. A tabela deve conter o nome do produto e seu respectivo preço. O programa também deve inserir alguns dados para teste.*

***Exercício 4*** - Modifique o programa do ***Exercício 3*** de forma a perguntar dois valores e listar todos os produtos com preços entre esses dois valores.

## Atualizando registros

Já sabemos como criar tabelas, inserir registros e fazer consultas simples. Vamos começar a usar o comando `UPDATE` para alterar nossos registros. Por exemplo, vamos alterar o registro com telefone de "Nilo" para "12345-6789":

```sql
UPDATE agenda SET telefone = "12345-6789" WHERE nome = 'Nilo'
```

A cláusula ``WHERE`` funciona como no comando ```SELECT```, ou seja, ela avalia uma expressão lógica que, quando verdadeira, inclui o registro na lista de registros a modificar. A segunda parte do comando ```UPDATE``` é cláusula ```SET```. Essa cláusula é usada para indicar o que fazer nos registros selecionados pela expressão do ```WHERE```. No exemplo, ```SET telefone = "12345-6789"``` muda o conteúdo do campo ```telefone``` para ```12345-6789```. O comando inteiro poderia ser lido como: atualize os registros da tabela agenda, alterando o telefone para "12345-6789" em todos os registros em que o campo ```nome``` é igual á "Nilo". Vejamos o programa:

```python
with sqlite3.connect("agenda.db") as conexão:
    with closing(conexão.cursor()) as cursor:
        cursor.execute(""" UPDATE agenda
                             SET telefone = '12345-6789'
                             WHERE nome = 'Nilo' """)
    conexão.commit()
# Verificar mudança de telefone no Exercício 5 acima ☝️
```

Nesse exemplo utilizamos constantes, logo não precisamos usar parâmetros. As mesmas regras que aprendemos para o comando ```SELECT``` se aplicam ao comando ```UPDATE```. Se os valores não forem constantes, você tem de utilizar parâmetros.

O comando ```UPDATE``` pode alterar mais de um registro de uma só vez. Faça uma cópia do arquivo ```agenda.db``` e experimente modificar o programa anterior, retirando a cláusula ```WHERE```:

```sql
UPDATE agenda SET telefone = "12345-6789"
```

Você verá que todos os registros foram modificados:
```
Nome: Nilo
Telefone: 12345-6789

Nome: João
Telefone: 12345-6789

Nome: André
Telefone: 12345-6789

Nome: Maria
Telefone: 12345-6789
```

Sem a cláusula ```WHERE``` , todos os registros serão alterados. Vamos utilizar a propriedade ```rowcount``` de nosso cursor para saber quantos registros foram alterados por nosso ```UPDATE```. Veja o programa com essas alterações:

```python
# Programa 6 Exemplo de UPDATE e com rowcount
with sqlite3.connect("agenda.db") as coxeção:
    with closing(conexão.cursor()) as cursor:
        cursor.execute("""
UPDATE agenda
SET telefone = '12345-6789'
""")
        print(f"Registros alterados: {cursor.rowcount}")
```


Não se esqueça de que, após modificar o banco de dados, precisamos chamar o método ```commit```, como fizemos ao inserir os registros. Caso nos esqueçamos, as alterações serão perdidas.

A propriedade ```rowcount``` é muito interessante para confirmarmos o resultado de comandos de atualização, como ```UPDATE```. Essa propriedade não funciona com ```SELECT```, retornando sempre -1. Por isso, no Programa 5, contamos os registros retornados por nosso ```SELECT``` em vez de usarmos ```rowcount```. No caso de ```UPDATE```, poderíamos fazer uma verificação de quantos registros seriam alterados antes de chamarmos o ```commit``` , como:

```python
with sqlite3.connect("agenda.db") as conexão:
    with closing(conexão.cursor()) as cursor:
        cursor.execute("""
UPDATE agenda
SET telefone = '12345-6789'
""")
        print(f"Registros alterados: {cursor.rowcount}")
        if cursor.rowcount == 1:
            conexão.commit()
            print("Alterações gravadas")
        else:
            conexão.rollback()
            print("Alterações abortadas")
````

***Exercício 5*** - Escreva um programa que aumente o preço de todos os produtos do banco de preços.db em 10%.

***Exercício 6***  - Escreva um programa que pergunte o nome do produto e um novo preço. Usando o banco preços.db, atualize o preço desse produto no banco de dados.

## Apagando registros

Além de inserir, consultar e alterar registros, podemos também apagá-los. O comando ```DELETE``` apaga registros com base em um critério de seleção, especificado na cláusula ```WHERE``` que já conhecemos. Faça outra cópia do arquivo ```agenda.db```. Copie o antigo banco de dados, com os registros antes de executarmos o Programa 6.

A sintaxe do comando ```DELETE``` é:

```sql
DELETE FROM agenda WHERE nome = 'Maria'
```

Ou seja, apague da tabela ```agenda``` todos os registros com nome igual a ```''Maria''``` :

```python
with sqlite3.connect("agenda.db") as conexão:
    with closing(conexão.cursor()) as cursor:
        cursor.execute("""
DELETE FROM agenda WHERE nome = 'Maria'
""")
        print(f"Registros apagados: {cursor.rowcount}")
        if cursor.rowcount == 1:
            conexão.commit()
            print("Alerações gravadas")
        else:
            conexão.roolback()
            print("Alterações abortadas")
```

Utilizamos o método ```rowcount``` para ter certeza de que estávamos apagando apenas um registro. Assim como no comando ```INSERT``` e ```UPDATE```, você precisa chamar ```commit``` para gravar as alterações ou ```roolback``` caso contrário.